# <b>Python for Data Analysis</b>
# 12. Introduction to Modeling Libraries in Python

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision = 4, suppress = True)
from pandas import Series, DataFrame
%matplotlib inline

## 12.1 Interfacing Between pandas and Model Code

In [2]:
data = pd.DataFrame({
    "x0": [1, 2, 3, 4, 5],
    "x1": [0.01, -0.01, 0.25, -4.1, 0.],
    "y": [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
df2 = pd.DataFrame(data.to_numpy(), columns = ["x0", "x1", "y"])
df2

,x0,x1,y
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [6]:
df3 = data.copy()
df3

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [7]:
df3["strings"] = list('abcde')
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [8]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [9]:
model_cols = ["x0", "x1"]

In [10]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [11]:
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [12]:
data["category"] = pd.Categorical(list("abaab"),
                                  categories = ["a", "b"])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [13]:
dummies = pd.get_dummies(data.category, prefix = "category",
                         dtype = float)

In [14]:
data_with_dummies = data.drop("category", axis = 1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1.0,0.0
1,2,-0.01,0.0,0.0,1.0
2,3,0.25,3.6,1.0,0.0
3,4,-4.10,1.3,1.0,0.0
4,5,0.00,-2.0,0.0,1.0


## 12.2 Creating Model Descriptions with Patsy

In [15]:
import patsy

In [16]:
data = pd.DataFrame({
    "x0": [1, 2, 3, 4, 5],
    "x1": [0.01, -0.01, 0.25, -4.1, 0.],
    "y": [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [17]:
y, X = patsy.dmatrices("y ~ x0 + x1", data)

In [18]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [19]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [20]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [21]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [22]:
patsy.dmatrices("y ~ x0 + x1 + 0", data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [23]:
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond = None)

In [24]:
coef

array([[ 0.3129],
       [-0.0791],
       [-0.2655]])

In [25]:
coef = pd.Series(coef.squeeze(), index = X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Data Transformations in Patsy Formulas

In [26]:
y, X = patsy.dmatrices("y ~ x0 + np.log(np.abs(x1) + 1)", data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [27]:
y, X = patsy.dmatrices("y ~ standardize(x0) + center(x1)", data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [28]:
new_data = pd.DataFrame({
    "x0": [6, 7, 8, 9],
    "x1": [3.1, -0.5, 0, 2.3],
    "y": [1, 2, 3, 4]})
new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [29]:
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [30]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Categorical Data and Patsy

In [31]:
data = pd.DataFrame({
    "key1": ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    "key2": [0, 1, 0, 1, 0, 1, 0, 0],
    "v1": [1, 2, 3, 4, 5, 6, 7, 8],
    "v2": [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [32]:
y, X = patsy.dmatrices("v2 ~ key2", data)
X

DesignMatrix with shape (8, 2)
  Intercept  key2
          1     0
          1     1
          1     0
          1     1
          1     0
          1     1
          1     0
          1     0
  Terms:
    'Intercept' (column 0)
    'key2' (column 1)

In [33]:
y, X = patsy.dmatrices("v2 ~ key2", data)
X

DesignMatrix with shape (8, 2)
  Intercept  key2
          1     0
          1     1
          1     0
          1     1
          1     0
          1     1
          1     0
          1     0
  Terms:
    'Intercept' (column 0)
    'key2' (column 1)

In [34]:
y, X = patsy.dmatrices("v2 ~ key2", data)
X

DesignMatrix with shape (8, 2)
  Intercept  key2
          1     0
          1     1
          1     0
          1     1
          1     0
          1     1
          1     0
          1     0
  Terms:
    'Intercept' (column 0)
    'key2' (column 1)

In [35]:
y, X = patsy.dmatrices("v2 ~ key2", data)
X

DesignMatrix with shape (8, 2)
  Intercept  key2
          1     0
          1     1
          1     0
          1     1
          1     0
          1     1
          1     0
          1     0
  Terms:
    'Intercept' (column 0)
    'key2' (column 1)

In [36]:
data["key2"] = data["key2"].map({0: "zero", 1: "one"})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [37]:
y, X = patsy.dmatrices("v2 ~ key1 + key2 + key1:key2", data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [38]:
y, X = patsy.dmatrices("v2 ~ key1 + key2 + key1:key2", data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## 12.3 Introduction to statsmodels

### Estimating Linear Models

In [39]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [42]:
rng = np.random.default_rng(seed = 12345)

def dnorm(mean, variance, size = 1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)
    
N = 100
X = np.c_[dnorm(0, 0.4, size = N),
          dnorm(0, 0.6, size = N),
          dnorm(0, 0.2, size = N)]
eps = dnorm(0, 0.1, size = N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [46]:
X[:5]

array([[-0.9005, -0.1894, -1.0279],
       [ 0.7993, -1.546 , -0.3274],
       [-0.5507, -0.1203,  0.3294],
       [-0.1639,  0.824 ,  0.2083],
       [-0.0477, -0.2131, -0.0482]])

In [47]:
y[:5]

array([-0.5995, -0.5885,  0.1856, -0.0075, -0.0154])

In [48]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.    , -0.9005, -0.1894, -1.0279],
       [ 1.    ,  0.7993, -1.546 , -0.3274],
       [ 1.    , -0.5507, -0.1203,  0.3294],
       [ 1.    , -0.1639,  0.824 ,  0.2083],
       [ 1.    , -0.0477, -0.2131, -0.0482]])

In [49]:
model = sm.OLS(y, X)

In [50]:
results = model.fit()

In [51]:
results.params

array([0.0668, 0.268 , 0.4505])

In [52]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Tue, 13 Feb 2024   Prob (F-statistic):                    2.66e-13
Time:                        19:54:42   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [53]:
data = pd.DataFrame(X, columns = ["col0", "col1", "col2"])

In [54]:
data["y"] = y

In [55]:
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [56]:
results = smf.ols("y ~ col0 + col1 + col2", data = data).fit()

In [57]:
results.params

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [58]:
results.tvalues

Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

In [59]:
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

### Estimating Time Series Processes

In [60]:
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [61]:
from statsmodels.tsa.ar_model import AutoReg

In [62]:
MAXLAGS = 5

In [63]:
model = AutoReg(values, MAXLAGS)

In [64]:
results = model.fit()

In [65]:
results.params

array([ 0.0235,  0.8097, -0.4287, -0.0334,  0.0427, -0.0567])

## 12.4 Introduction to scikit-learn

In [67]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')

In [68]:
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [69]:
test.head(4)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S


In [70]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [71]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [73]:
impute_value = train["Age"].median()
impute_value

28.0

In [74]:
train["Age"] = train["Age"].fillna(impute_value)

In [75]:
test["Age"] = test["Age"].fillna(impute_value)

In [76]:
train["IsFemale"] = (train["Sex"] == "female").astype(int)
test["IsFemale"] = (test["Sex"] == "female").astype(int)

In [79]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1


In [81]:
test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0


In [82]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [84]:
X_train = train[predictors].to_numpy()
X_test = test[predictors].to_numpy()
y_train = train["Survived"].to_numpy()

In [85]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [86]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

In [87]:
from sklearn.linear_model import LogisticRegression

In [88]:
model = LogisticRegression()

In [89]:
model.fit(X_train, y_train)

LogisticRegression()

In [90]:
y_predict = model.predict(X_test)

In [91]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

Way to calculate an accuracy percentage or some other error metric:

In [93]:
# (y_true == y_predict).mean()

In [94]:
from sklearn.linear_model import LogisticRegressionCV

In [95]:
model_cv = LogisticRegressionCV(Cs = 10)

In [96]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

In [97]:
from sklearn.model_selection import cross_val_score

In [98]:
model = LogisticRegression(C = 10)

In [99]:
scores = cross_val_score(model, X_train, y_train, cv = 4)
scores

array([0.7758, 0.7982, 0.7758, 0.7883])